In [74]:
!pip install nltk
#!pip install spacy
!python -m nltk.downloader averaged_perceptron_tagger
!pip install --upgrade nltk
!pip install transformers
!pip install torch torchvision
!pip install --upgrade diffusers accelerate transformers
# !pip install controlnet_aux


<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [76]:
import nltk
from nltk.corpus import wordnet as wn
from nltk.corpus import gutenberg
#from nltk import pos_tag
import random
#import spacy
from transformers import pipeline


# PROMPT GENERATION

## Random words

In [77]:
# Download the WordNet dataset
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/jovyan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [4]:
# Backups
nouns = list(wn.all_synsets('n'))
adjectives = list(wn.all_synsets('a'))
verbs = list(wn.all_synsets('v'))
random_noun = random.choice(nouns).lemmas()[0].name()
random_adjective = random.choice(adjectives).lemmas()[0].name()
random_verb = random.choice(verbs).lemmas()[0].name()

## Random words from famous artists

In [3]:
nltk.download('gutenberg')

[nltk_data] Downloading package gutenberg to /home/jovyan/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


True

In [4]:
book_complete = gutenberg.words('austen-emma.txt')
austen_emma = ' '.join([word for word in book_complete if word.isalpha()])

In [5]:
nltk.data.path.append("/home/jovyan/nltk_data")  # Ensure the correct path is used
nltk.download('punkt')

nltk.download('averaged_perceptron_tagger', download_dir='/home/jovyan/nltk_data')
nltk.download('punkt_tab')  # This will download the missing tokenizer files

[nltk_data] Downloading package punkt to /home/jovyan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/jovyan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package punkt_tab to /home/jovyan/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [6]:
pipe = pipeline("token-classification", model="vblagoje/bert-english-uncased-finetuned-pos", device=0)

Some weights of the model checkpoint at vblagoje/bert-english-uncased-finetuned-pos were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
def prompt_generation(text):

    def chunk_text(text, max_length=512):
        words = text.split()  # Split text into words
        chunks = []
        current_chunk = []
    
        # Iterate over words and form chunks
        for word in words:
            if len(" ".join(current_chunk + [word])) <= max_length:
                current_chunk.append(word)
            else:
                chunks.append(" ".join(current_chunk))
                current_chunk = [word]
                
        # Add the last chunk if it exists
        if current_chunk:
            chunks.append(" ".join(current_chunk))
        return chunks
        
    # Chunk the input text
    chunks = chunk_text(text)
    
    # Initialize lists to store words by POS
    nouns = []
    verbs = []
    adjectives = []
    
    # Process each chunk and classify POS
    for chunk in chunks:
        word_classification = pipe(chunk)
        
        for word in word_classification:
            pos = word.get('entity')
            word_text = word.get('word')
            
            if pos == 'NOUN':
                nouns.append(word_text)
            elif pos == 'VERB':
                verbs.append(word_text)
            elif pos == 'ADJ':
                adjectives.append(word_text)
                
    # Filter out tokenized words starting with ##
    nouns = [word for word in nouns if not word.startswith("##")]
    verbs = [word for word in verbs if not word.startswith("##")]
    adjectives = [word for word in adjectives if not word.startswith("##")]

    # Check if lists are non-empty before selecting random words
    prompt_components = []
    
    if adjectives:
        prompt_components.append(random.choice(adjectives))
    if nouns:
        prompt_components.append(random.choice(nouns))
    if verbs:
        prompt_components.append(random.choice(verbs))
    if adjectives:
        prompt_components.append(random.choice(adjectives))
    if nouns:
        prompt_components.append(random.choice(nouns))
    
    # Join the prompt components
    prompt = ' '.join(prompt_components)
    
    return prompt


In [9]:
random.seed(15)

prompt = prompt_generation(austen_emma)
prompt

'contrary subject hearing small law'

# IMAGE GENERATION

## N°1

In [139]:
import requests
from PIL import Image, ImageOps, ImageDraw
from io import BytesIO
from diffusers import StableDiffusionPipeline
#from diffusers import StableDiffusionUpscalePipeline
from diffusers import StableDiffusionImg2ImgPipeline
import torch

In [140]:
#generator = torch.Generator().manual_seed(42) # seed
txt_2_img_pipe = StableDiffusionPipeline.from_pretrained("runwayml/stable-diffusion-v1-5", use_safetensors=True)
txt_2_img_pipe.to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

StableDiffusionPipeline {
  "_class_name": "StableDiffusionPipeline",
  "_diffusers_version": "0.31.0",
  "_name_or_path": "runwayml/stable-diffusion-v1-5",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": true,
  "safety_checker": [
    "stable_diffusion",
    "StableDiffusionSafetyChecker"
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

In [145]:
torch.manual_seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

torch.cuda.synchronize() 

In [146]:
# HARD CODED TO DELETE
prompt = 'contrary subject hearing small law'
#negative_prompt = 'blurry, text, letters, typography, writing, low resolution, watermark, patterns, distorted face, bad anatomy, uncanny, extra fingers, missing fingers, missing faces, pixelated, deformed'
negative_prompt = 'blurry, text, letters, typography, writing, low resolution, watermark, patterns, distorted face, bad anatomy, uncanny, extra fingers, missing fingers, missing faces, pixelated, deformed, no text'
positive_prompt = 'clean lines, well-balanced composition'

In [148]:
height = 1080/len(words)  # Set height
width = 1080  # Set width

words = prompt.split()
image_1 = txt_2_img_pipe(words[0] + positive_prompt, negative_prompt=negative_prompt, guidance_scale=13, num_inference_steps= 80, height=height, width=width).images[0]
image_1.save('image_1.png')

# Show the image
image_1
print(words[0])


  0%|          | 0/80 [00:00<?, ?it/s]

contrary


## Extend picture n°2 and followings

In [149]:
# Load the pre-trained model
img2img_pipe = StableDiffusionImg2ImgPipeline.from_pretrained("stabilityai/stable-diffusion-2-1-base", torch_dtype=torch.float16)
img2img_pipe.to('cuda')


Loading pipeline components...:   0%|          | 0/6 [00:00<?, ?it/s]

StableDiffusionImg2ImgPipeline {
  "_class_name": "StableDiffusionImg2ImgPipeline",
  "_diffusers_version": "0.31.0",
  "_name_or_path": "stabilityai/stable-diffusion-2-1-base",
  "feature_extractor": [
    "transformers",
    "CLIPImageProcessor"
  ],
  "image_encoder": [
    null,
    null
  ],
  "requires_safety_checker": false,
  "safety_checker": [
    null,
    null
  ],
  "scheduler": [
    "diffusers",
    "PNDMScheduler"
  ],
  "text_encoder": [
    "transformers",
    "CLIPTextModel"
  ],
  "tokenizer": [
    "transformers",
    "CLIPTokenizer"
  ],
  "unet": [
    "diffusers",
    "UNet2DConditionModel"
  ],
  "vae": [
    "diffusers",
    "AutoencoderKL"
  ]
}

### GIF PREPARATION - MODIFY IMAGE WITH NEXT WORD. todo modify format if for instagram
Input = previous image, next prompt word

In [31]:
words = prompt.split()

strength = 0.8  # How much the original image is preserved
guidance_scale = 7 # How closely the image should match the prompt

# Set the initial image as the starting point
current_image = Image.open("image_1.png").convert("RGB")

# Iterate over each word in the prompt
for i, word in enumerate(words[1:], start=1):

    output = img2img_pipe(
        prompt=word + positive_prompt,  
        negative_prompt=negative_prompt,
        image=current_image, 
        strength=strength,
        guidance_scale=guidance_scale
    ).images[0]

    # Save the current output image
    output_filename = f"output_image_{i+1}.jpg"
    output.save(output_filename)
    
    # Use the current output image as the input for the next iteration
    current_image = output

    print(f"Saved image: {output_filename}")

  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_image_2.jpg


  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_image_3.jpg


  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_image_4.jpg


  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_image_5.jpg


### Gif maker iterating with the entire prompt
Could be with several different prompt as well and/or poems

In [28]:
words = prompt.split()

strength = 0.8  # How much the original image is preserved
guidance_scale = 7 # How closely the image should match the prompt

# Set the initial image as the starting point
current_image = Image.open("image_1.png").convert("RGB")

# Iterate over each word in the prompt
for i in range(len(words)):

    output = img2img_pipe(
        prompt=prompt + positive_prompt,  
        negative_prompt=negative_prompt,
        image=current_image, 
        strength=strength,
        guidance_scale=guidance_scale
    ).images[0]

    # Save the current output image
    output_filename = f"output_full_prompt_{i+1}.jpg"
    output.save(output_filename)
    
    # Use the current output image as the input for the next iteration
    current_image = output

    print(f"Saved image: {output_filename}")

  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_full_prompt_1.jpg


  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_full_prompt_2.jpg


  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_full_prompt_3.jpg


  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_full_prompt_4.jpg


  0%|          | 0/40 [00:00<?, ?it/s]

Saved image: output_full_prompt_5.jpg


### EXTEND IMAGE WITH NEXT WORD.
Input = previous image, next prompt word

In [162]:
def expand_img(input_img, word):
    input_img = Image.open(input_img).convert("RGB")
    expanded_size = ( input_img.width, input_img.height)
    top_part = input_img.crop((0, 0, input_img.width, input_img.height // 5))
    top_part = top_part.resize(expanded_size)

    expanded_image = img2img_pipe(
    prompt=word + positive_prompt,
    negative_prompt=negative_prompt,
    image=top_part,
    strength=0.75,
    guidance_scale=7.5).images[0]
    return expanded_image



In [165]:
input_img = 'image_1.png'
for i in words[1:]:
    expanded = expand_img(input_img, i)
    expanded.save(f"expanded_{i}.png")

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

In [90]:
inpaint_pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting",
    torch_dtype=torch.float16
).to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [95]:
words = prompt.split()
words

['contrary', 'subject', 'hearing', 'small', 'law']

In [101]:
from PIL import Image, ImageDraw

# Load your input image
original_image = Image.open("image_1.png").convert("RGB")

# Iteratively expand the image upward
for idx, word in enumerate(words):
    # Define the new canvas size (extend height upward)
    new_height = original_image.height + 256  # Add 256 pixels at the top
    new_width = original_image.width
    canvas = Image.new("RGB", (new_width, new_height), (255, 255, 255))  # Create larger canvas
    canvas.paste(original_image, (0, 256))  # Place the current image at the bottom

    # Sample content from the top of the original image to guide blending
    top_sample = original_image.crop((0, 0, new_width, 50))  # Take a slice from the top

    # Create a mask for the blank area (the top part of the canvas)
    mask = Image.new("L", (new_width, new_height), 0)  # Black (preserve original)
    draw = ImageDraw.Draw(mask)
    draw.rectangle([0, 0, new_width, 256], fill=255)  # White (generate area at top)

    # Use the current image and sampled content as the base for generation
    prompt = f"{word} blended with sky and texture from image"
    blended_prompt = f"{word} inspired by the continuation of the scene"

    # Generate new content for the blank area
    result = inpaint_pipe(
        prompt=prompt + positive_prompt,
        negative_prompt=negative_prompt,
        image=canvas,
        mask_image=mask,
        strength=0.75,  # Lower strength to preserve context
        guidance_scale=7.5
    ).images[0]

    # Save intermediate results (optional)
    intermediate_filename = f"expanded_upward_step_{idx + 1}.jpg"
    result.save(intermediate_filename)

    # Update the original image for the next iteration with the new expanded image
    original_image = result  # Now use the expanded result as the base for the next step

# Save the final expanded image
original_image.save("final_expanded_upward_image3.jpg")


  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

  0%|          | 0/37 [00:00<?, ?it/s]

In [61]:
# Pick an image from an image dataset rather than generating it? and make a collage?

In [55]:
# ETERNAL RIGHT SCROLL OR BOTTOM, ETC.
# Prompt[:1] --> image1 --> right part of image1 + prompt[1:2] + style of image1 and noise/random --> image2 --> right part of image2 + prompt[2:3] + style of image 1...

In [ ]:
# ZOOM OUT OF ZOOM IN
# prompt[:1] --> image1 --> image1 + zoom out (bigger picture) --> 

In [88]:
from diffusers import StableDiffusionInpaintPipeline
from PIL import Image
import torch

# Load the pipeline
pipe = StableDiffusionInpaintPipeline.from_pretrained(
    "runwayml/stable-diffusion-inpainting", 
    torch_dtype=torch.float16
).to("cuda")

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

An error occurred while trying to fetch /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae: Error no file named diffusion_pytorch_model.safetensors found in directory /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/vae.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.
An error occurred while trying to fetch /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet: Error no file named diffusion_pytorch_model.safetensors found in directory /home/jovyan/.cache/huggingface/hub/models--runwayml--stable-diffusion-inpainting/snapshots/8a4288a76071f7280aedbdb3253bdb9e9d5d84bb/unet.
Defaulting to unsafe serialization. Pass `allow_pickle=False` to raise an error instead.


In [137]:
torch.cuda.empty_cache()
%reset -f